# pybithumb

In [1]:
# pybithumb 설치 (Anaconda Prompt)
# 처음 설치: pip install pybithumb
# 업데이트 설치: pip install –U pybithumb
# https://github.com/sharebook-kr/pybithumb

In [2]:
# 시세 다운로드
import pybithumb

df = pybithumb.get_ohlcv("BTC")
df.to_excel("BTC.xlsx")
# root directory에 저장

In [3]:
# 현재가 얻기
# pybithumb.get_current_price(ticker)

import pybithumb
import time

cur_price = pybithumb.get_current_price("BTC")
print(cur_price, type(cur_price))

11644000.0 <class 'float'>


In [5]:
# 주기적으로 현재가 출력
while True:
    cur_price = pybithumb.get_current_price("BTC")
    print(cur_price)
    time.sleep(0.5)

11644000.0
11642000.0
11642000.0
11642000.0
11642000.0


In [6]:
# 최근 5일 시세만 출력
df = pybithumb.get_ohlcv("BTC")
print(df.tail())    

                           open       close        high         low  \
BTC                                                                   
2020-02-19 00:00:00  11421000.0  11855000.0  11940000.0  11418000.0   
2020-02-20 00:00:00  11858000.0  11460000.0  11959000.0  11350000.0   
2020-02-21 00:00:00  11460000.0  11536000.0  11613000.0  11350000.0   
2020-02-22 00:00:00  11536000.0  11489000.0  11591000.0  11397000.0   
2020-02-23 21:00:00  11489000.0  11644000.0  11800000.0  11442000.0   

                          volume  
BTC                               
2020-02-19 00:00:00  4498.708652  
2020-02-20 00:00:00  5304.663821  
2020-02-21 00:00:00  2789.747651  
2020-02-22 00:00:00  1532.794772  
2020-02-23 21:00:00  1891.742822  


In [7]:
# 변동성 돌파 전략 함수 (진입 목표가 계산)
def cal_target(k=0.5):
    df = pybithumb.get_ohlcv("BTC")
    yesterday = df.iloc[-2]
    today = df.iloc[-1]

    yesterday_high = yesterday['high']
    yesterday_low = yesterday['low']
    today_open = today['open']

    yesterday_range = yesterday_high - yesterday_low
    target = today_open + yesterday_range * k
    return target

target = cal_target()
print(target)

11586000.0


In [8]:
# 매일 자정 시각을 리턴하는 함수(00:00:00~00:00:10)

import datetime

def make_times():
    now = datetime.datetime.now() # 현재 시각
    tomorrow = now + datetime.timedelta(days=1) # 현재 시각 + 1D
    midnight = datetime.datetime(year=tomorrow.year,
                                 month=tomorrow.month,
                                 day=tomorrow.day,
                                 hour=0,
                                 minute=0,
                                 second=0)

    midnight_after_10secs = midnight + datetime.timedelta(seconds=10)
    return midnight, midnight_after_10secs # 튜플 하나로 간주하여 리턴

print(make_times())

(datetime.datetime(2020, 2, 24, 0, 0), datetime.datetime(2020, 2, 24, 0, 0, 10))


In [9]:
# 프로그램 기본 구조

setup_time1, setup_time2 = make_times()
target = cal_target(k=0.5)

while True:
    now = datetime.datetime.now()

    if setup_time1 < now < setup_time2:
        target = cal_target(k=0.5)
        setup_time1, setup_time2 = make_times()
        time.sleep(10)

    cur_price = pybithumb.get_current_price("BTC")
    print("{} 현재가 {} 목표가 {}".format(now, cur_price, target))

    time.sleep(0.5)

2020-02-23 21:40:29.138057 현재가 11643000.0 목표가 11586000.0
2020-02-23 21:40:29.731485 현재가 11643000.0 목표가 11586000.0
2020-02-23 21:40:30.346914 현재가 11643000.0 목표가 11586000.0
2020-02-23 21:40:30.935322 현재가 11643000.0 목표가 11586000.0
2020-02-23 21:40:31.555780 현재가 11643000.0 목표가 11586000.0


In [10]:
# API Key 입력
key = ""
secret = ""
# Input your private key

bithumb = pybithumb.Bithumb(key, secret)
print(bithumb)

In [11]:
# 잔고 조회
bithumb = pybithumb.Bithumb(key, secret)
balance = bithumb.get_balance("ETH")
print(balance)

(9e-06, 0.0, 0.95795, 0.0)


In [12]:
krw_balance = balance[2]
print(krw_balance)

0.95795


In [13]:
# 매수 호가 조회
import pybithumb

orderbook = pybithumb.get_orderbook("BTC")
bids = orderbook['bids'] # 매도 호가 : 'asks'

for bid in bids:
    print(bid['quantity'], bid['price'])

0.375 11643000.0
0.4935 11642000.0
0.0125 11641000.0
0.693 11640000.0
0.299 11639000.0


In [14]:
# 지정가 매수 주문
ret = bithumb.buy_limit_order("XRP", 500, 10)
print(ret)

{'status': '5600', 'message': '주문량이 사용가능 KRW을 초과하였습니다.'}


In [15]:
# 주문 취소
order_desc = bithumb.buy_limit_order("BTC", 11000000, 1)
status = bithumb.cancel_order(order_desc)

In [16]:
# 시장가 매수 주문
ret = bithumb.buy_market_order("XRP", 10)
print(ret)

{'status': '5600', 'message': '주문량이 사용가능 KRW을 초과하였습니다.'}


In [17]:
# 시장가 매도 주문
# 1) 수량
balance = bithumb.get_balance("XRP")
xrp_unit = balance[0]

# 2) 매도
ret = bithumb.sell_market_order("XRP", xrp_unit)
print(ret)

{'status': '5500', 'message': 'Invalid Parameter'}


In [18]:
# 예외 처리
mydict = {"BTC": 7300000, "XRP": 560}
print(mydict["BTC"])

7300000


In [19]:
try:
    print(mydict["BTG"])
except:
    print("error")

# https://wikidocs.net/30

error


In [20]:
# 매수 함수 구현 : 
def try_buy(now, cur_price, target_price):
    krw_balance = bithumb.get_balance("BTC")[2]
    
    # 최우선 매도호가의 가격 얻기 
    orderbook = pybithumb.get_current_price("BTC")
    asks = orderbook['asks']
    price0 = asks[0]['price']
    unit = krw_balance/price0
    
    if cur_price >= target_price:
        bithumb.buy_market_order("BTC", unit)